いろいろ確認

- データをグループ化
- データを可視化

In [ ]:
import pandas as pd
import altair as alt
import titanite as ti

print(f"Pandas {pd.__version__}")
print(f"Altair {alt.__version__}")
print(f"Titanite {ti.__version__}")

In [ ]:
f_cfg = "../sandbox/config.toml"
f_csv = "../data/test_data/prepared_data.csv"
d = ti.Data(read_from=f_csv, load_from=f_cfg)
config = d.config()
data = d.read()
# data = d.read()
# data


In [ ]:
n_headers = d.numerical_headers
c_headers = d.categorical_headers

各質問の記述統計

- 数値型のカラムに対して、平均、標準偏差、最小値、最大値などを計算する
- カテゴリカルなカラムに対して、頻度とパーセンテージを取得する
- 自由記述のカラムは除外する

In [ ]:
n_stats = data[n_headers].describe().T
c_stats = data[c_headers].apply(lambda x: x.value_counts(dropna=False)).T
n_stats
# c_stats

性別と感情分析（極性と主観度）

- カテゴリ型と数値型は、直接的な相関を計算できない
- 性別ごとに極性と主観度の平均値を計算して比較する
- 箱ひげ図を使って、性別ごとの極性と主観度の分布を可視化する

In [ ]:
# Compute the mean values
g = ["q02"]
headers = ["q15_polarity", "q15_subjectivity"]
mean_values = data.groupby(g)[headers].describe().T
mean_values

In [ ]:
alt.Chart(data).mark_boxplot().encode(
        alt.X("q03_subregional"),
        alt.Y("q10"),
).properties(
    title="Boxplot of q15_polarity by gender",
    width=800,
).interactive()

In [ ]:
f_q15 = "../data/test_data/comment/q15.csv"
q15 = pd.read_csv(f_q15)
q15.groupby(g)[headers].mean()

In [ ]:
alt.Chart(q15).mark_boxplot().encode(
        alt.X("q02"),
        alt.Y("q15_polarity"),
).properties(
    title="Boxplot of q15_polarity by gender",
    width=500,
)

alt.Chart(q15).mark_boxplot().encode(
        alt.X("q02"),
        alt.Y("q15_subjectivity"),
).properties(
    title="Boxplot of q15_subjectivity by gender",
    width=500,
)

In [ ]:
def group_data(data: pd.DataFrame, group: list) -> pd.DataFrame:
    c = "response"
    grouped = data.groupby(group)[c].sum().reset_index()
    return grouped

In [ ]:
group_data(data, ["q1", "q2", "q3"]).sort_values("response", ascending=False)

In [ ]:
def histogram(data: pd.DataFrame, x: str, color: str):
    y = "response"
    h = alt.Chart(data).mark_bar().encode(
        alt.X(x),
        alt.Y(y),
        alt.Color(color),
    ).properties(
        width = 500,
    )
    return h    

In [ ]:
def heatmap(data: pd.DataFrame, x: str, y: str, color: str):
    hm = alt.Chart(data).mark_rect().encode(
        alt.X(x),
        alt.Y(y),
        alt.Color(color),
    ).properties(
        width = 500,
    )
    return hm

In [ ]:
x = "q2"
y = "q6"
z = "response"

g = [x, y]
gdata = group_data(data, g)
# hi = histogram(gdata, x, y)
# hm = heatmap(gdata, x, y, color="response")